# _ prepare colab envirment _

In [1]:
# check running accerleration

## import
import os
import pprint
import tensorflow as tf

## check gpu
print('---gpu check---')
print(format(tf.test.gpu_device_name()))
### ' ' means no gpu
### '/device:GPU:0' means use gpu

##  check tpu
print('\n---tpu check---')
try:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

    with tf.Session(tpu_address) as session:
        devices = session.list_devices()
        print('TPU devices:')
        pprint.pprint(devices)
except KeyError:
    print('/tpu:disabled')
else:
    pass
finally:
    pass

print('\n_check complete_')

# mount google drive

## install drive lib
!pip install -U -q PyDrive
## mount drive
from google.colab import drive
drive.mount('/content/drive/')

---gpu check---
/device:GPU:0

---tpu check---
/tpu:disabled

_check complete_
     |████████████████████████████████| 993kB 5.0MB/s 
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
# set working dir

## import
import os

## set project working folder
tfolder = '/content/drive/My Drive/ml/project/toxic/'
pfolder = '/content/drive/My Drive/ml/nlppre/'

## change dir
os.chdir(tfolder)

## check position
print('\n---position---')
!pwd

## check file
print('\n---folder files---')
!ls -al


---position---
/content/drive/My Drive/ml/project/toxic

---folder files---
total 837035
-rw------- 1 root root  21693763 Apr 29 03:32 s1_sub.csv
-rw------- 1 root root 656423264 May 10 21:37 s2_1_best.h5
-rw------- 1 root root      8071 May 10 21:42 s2_1_model.yaml
-rw------- 1 root root  21569176 May 10 21:43 s2_1_sub.csv
-rw------- 1 root root      4525 Apr 29 08:25 s2_model.yaml
-rw------- 1 root root  21985027 Apr 29 08:25 s2_sub.csv
-rw------- 1 root root   6279782 Apr 24 06:16 sample_submission.csv
-rw------- 1 root root  60354593 Apr 24 06:16 test.csv
-rw------- 1 root root  68802655 Apr 24 06:17 train.csv


In [0]:
# prepare prefiles(if need)

## change dir
os.chdir(pfolder)

## unzip file if need
# !unzip -qq glove.6B.zip

## check position
print('\n---position---')
!pwd

## check file
print('\n---folder files---')
!ls -al

## change dir
os.chdir(tfolder)

## check position
print('\n---position---')
!pwd

## check file
print('\n---folder files---')
!ls -al


---position---
/content/drive/My Drive/ml/nlppre

---folder files---
total 6527539
-rw------- 1 root root 1037962819 Apr 28 03:27 glove.6B.300d.txt
-rw------- 1 root root 5646236541 Oct 24  2015 glove.840B.300d.txt

---position---
/content/drive/My Drive/ml/project/toxic

---folder files---
total 837035
-rw------- 1 root root  21693763 Apr 29 03:32 s1_sub.csv
-rw------- 1 root root 656423264 May 10 21:37 s2_1_best.h5
-rw------- 1 root root      8071 May 10 21:42 s2_1_model.yaml
-rw------- 1 root root  21569176 May 10 21:43 s2_1_sub.csv
-rw------- 1 root root      4525 Apr 29 08:25 s2_model.yaml
-rw------- 1 root root  21985027 Apr 29 08:25 s2_sub.csv
-rw------- 1 root root   6279782 Apr 24 06:16 sample_submission.csv
-rw------- 1 root root  60354593 Apr 24 06:16 test.csv
-rw------- 1 root root  68802655 Apr 24 06:17 train.csv


In [0]:
# check env

## keras
import keras
print('\n---keras version: {}---'.format(keras.__version__))


---keras version: 2.2.4---


Using TensorFlow backend.


# _ solution brief _

- project - Toxic Comment Classification
- data - kaggle
- score - 0.96344 (kaggle private)
- solution2_2 - glove300b+CNN
- solution brief - GloVe6b300D, CNN 7 layer
- finding - 基本是最好得分了（又一个更好一点）改进思路：
    - embedding 使用 fasttext
    - model 使用 lstm
    - dropout 0.5
    - solution2 关闭

# I - prepare data

In [0]:
# 导入基础库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pprint import pprint as p

# 导入机器学习库
from sklearn.model_selection import train_test_split

# 导入深度学习库
from sklearn.datasets import load_files

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential

from keras.layers import Embedding, Flatten, Dense, Dropout, SpatialDropout1D
from keras.layers import Conv1D, MaxPool1D, BatchNormalization

from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from keras.models import model_from_json

# 设置参数显示长文本
pd.options.display.max_colwidth = 500

# 设置行内显示
%matplotlib inline

In [0]:
# import files

## read test
file = 'test.csv'
filepath = tfolder+file
filepath
test = pd.read_csv(filepath)

## read train
file = 'train.csv'
filepath = tfolder+file
filepath
train = pd.read_csv(filepath)

In [0]:
# check files

test.head(1)
## 第6行是良好评论数据
## 官方test_label中 -1 是可能的选择，0为非攻击性语言

,id,comment_text
0,00001cee341fdb12,"Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,"


In [0]:
train.head(1)
## 注意第7行是train的目标数据处理结果
## 将涉及到的负面类型通过1来标记

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,0,0,0,0,0


# II - word embedding

In [0]:
# get comment
train_comment = train.comment_text
test_comment = test.comment_text

In [0]:
# get lable
train_label = train.iloc[:,2:]
train_label.head(1)

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0


In [0]:
# check data
p(train_comment[0])
print(train_comment.shape)
p(test_comment[0])
print(test_comment.shape)
print(train_label.shape)

('Explanation\n'
 'Why the edits made under my username Hardcore Metallica Fan were reverted? '
 "They weren't vandalisms, just closure on some GAs after I voted at New York "
 "Dolls FAC. And please don't remove the template from the talk page since I'm "
 'retired now.89.205.38.27')
(159571,)
("Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and "
 'hating you sad mofuckas...i should bitch slap ur pethedic white faces and '
 'get you to kiss my ass you guys sicken me. Ja rule is about pride in da '
 'music man. dont diss that shit on him. and nothin is wrong bein like tupac '
 'he was a brother too...fuckin white boys get things right next time.,')
(153164,)
(159571, 6)


In [0]:
# --- 6B 300D ---
# 根据测试，300D的成绩明显好于50D
# 也说明合适的维度非常重要

# 向量化词

max_feature = 10000
max_lenth = 100

## prepare tokenizer
t = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', num_words=max_feature )
## https://keras.io/preprocessing/text/
## 去掉奇怪字符、lower可以提高一点分数
t.fit_on_texts(train_comment)
## 这里是对输入句子进行拆词
## 在这个数据里，如果输入了1000句（测试文件中的train_short)，结果是10007个
## 如果输出的话在1000之后会有个...
## 全部输入的话是21万多
## 可以使用 num_words 做限制（这里没有生效，后续研究）

vocab_size = len(t.word_index) + 1
## 根据上面设置 vocab_size

## integer encode the documents
encoded_train = t.texts_to_sequences(train_comment)
encoded_test = t.texts_to_sequences(test_comment)

## pad documents to a max length of 100 words
max_length = 300

## max_length 就是每个commet要处理的单词数
padded_train = pad_sequences(encoded_train, maxlen=max_length, padding='post')
padded_test = pad_sequences(encoded_test, maxlen=max_length, padding='post')

In [0]:
# check padded
print(padded_train.shape)
p(padded_train[99])

print(padded_test.shape)
p(padded_test[100])
## 可以看出来，100个词的 padded 数据，没有的用0填充了

(159571, 300)
array([   6,   40,   33,   42,  173,  282,  145,   89,   26,   22,    6,
         96,    5,  672,  884,   16,  528,    2,   33,   57,    4,   18,
         57, 2292,    8,   39,   73,  371,    4,    6,  361,    2,   16,
        101, 1463,   21,  567,   37,    6,  292,   18,    5,  672,   50,
         96,   48,   11, 1018,  439, 3246,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0, 

In [0]:
# check word list
len(t.word_index)
## 注意这里是词统计完的计数（输入文本中有多少词）
## 与 max_length 无关，因为 pad_sequences 是后发生的

210337

# III - modeling s2.2

In [0]:
# 创建 embedding 层（300D）
## 840b 时发现处理大文件要使用 with open，代码更新（之前的报错）

## 初始化参数
embed_demention = 300
embeddings_index = dict( )

## 读取文件
file = 'glove.840B.300d.txt'
filepath = pfolder+file

with open(filepath,encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Loaded %s word vectors.' % len(embeddings_index))

## 创建嵌入矩阵
embedding_matrix = np.zeros((vocab_size, embed_demention))

## 注意这里要和嵌入矩阵的维度300相同
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Loaded 2196016 word vectors.


In [0]:
# check embedding_matrix
embedding_matrix.shape

(210338, 300)

In [0]:
# update2.1
## test=0.1

# split train and val

x_train, x_val , y_train, y_val = train_test_split(padded_train, train_label, test_size=0.1, shuffle=True, random_state=42)

In [0]:
# update2.1
## drop regulation
## keep only one cnn

# update2.2
## drop 0.2
## use spatial dropout

basefilter=64
kernel=3

# model
model = Sequential()

## 设置embedding层
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=300, trainable=False)
model.add(e)

model.add(SpatialDropout1D(0.2))
model.add(BatchNormalization())

##2.1add activation
model.add(Conv1D(basefilter, kernel, padding='same', activation='relu'))
model.add(MaxPool1D())

model.add(Conv1D(basefilter*2, kernel, padding='same', activation='relu'))
model.add(MaxPool1D())

model.add(Conv1D(basefilter*4, kernel, padding='same', activation='relu'))
model.add(MaxPool1D())

model.add(Conv1D(basefilter*8, kernel, padding='same', activation='relu'))
model.add(MaxPool1D())

model.add(Conv1D(basefilter*16, kernel, padding='same', activation='relu'))
model.add(MaxPool1D())

model.add(Conv1D(basefilter*32, kernel, padding='same', activation='relu'))
model.add(MaxPool1D())

model.add(Conv1D(basefilter*64, kernel, padding='same', activation='relu'))
model.add(MaxPool1D())

model.add(Flatten())
model.add(Dense(6, activation='sigmoid'))
# 6分类 Dense为6

## compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
## summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 300, 300)          63101400  
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 300, 300)          0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 300, 300)          1200      
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 300, 64)           57664     
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 150, 64)           0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 150, 128)          24704     
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 75, 128)           0         
__________

In [0]:
# set name
name = 's2_2'

mname = name+'_model.yaml'
wname = name+'_weight.h5'
sname = name+'_sub.csv'
bname = name+'_best.h5'

In [0]:
# update2.1

from keras.callbacks import EarlyStopping, ModelCheckpoint

# Set callback functions to early stop training and save the best model so far
callbacks = [EarlyStopping(monitor='val_loss', patience=2),
             ModelCheckpoint(filepath=bname, monitor='val_loss', save_best_only=True)]

In [0]:
# fit the model
model.fit(x_train, y_train, batch_size=64, epochs=20, validation_data=(x_val, y_val), callbacks=callbacks, verbose=2)

Train on 143613 samples, validate on 15958 samples
Epoch 1/20


# IV - conclusion

- eporch 20 很快就开始抖动了，设定了 earystoping
- 成绩不高应该是模型问题，单纯cnn的方式需要改进
- 发现增加层数（每层 filter*2）会提高一点成绩

In [0]:
## save model
model_yaml = model.to_yaml()
with open(mname, "w") as yaml_file:
    yaml_file.write(model_yaml)
    
print('- model saved as {}'.format(mname))
    
## save weights(200M+ if needed)
#model.save_weights(wname)
#print('- weight saved as {}'.format(wname))

## save submission
y_target = model.predict(padded_test)
submission = pd.read_csv('sample_submission.csv')
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_target
submission.to_csv(sname, index=False)
print('- submission saved as {}'.format(sname))

#_ kaggle _

In [2]:
# set kaggle(run once)

## make dir
!mkdir ~/.kaggle

## copy token
!cp '/content/drive/My Drive/ml/kaggle.json' ~/.kaggle/

## check file
!ls ~/.kaggle

kaggle.json


In [3]:
# search for competition(data the sameway)
# !kaggle competitions list -s toxic

# check competition score
!kaggle competitions submissions -c jigsaw-toxic-comment-classification-challenge

fileName           date                 description                                                                                   status    publicScore  privateScore  
-----------------  -------------------  --------------------------------------------------------------------------------------------  --------  -----------  ------------  
s2_2_sub.csv       2019-05-10 23:28:39  sub2_2:glove840B+CNN_update                                                                   complete  0.96766      0.96938       
s2_2_sub.csv       2019-05-10 23:04:25  sub2_2:glove840B+CNN_update                                                                   complete  0.95929      0.96174       
s2_2_sub.csv       2019-05-10 22:47:04  sub2_2:glove840B+CNN_update                                                                   complete  0.96413      0.96396       
s2_1_sub.csv       2019-05-10 21:43:48  sub2_1:glove6B+CNN_update                                                                     comple

In [4]:
# submit(once)
!kaggle competitions submit -c jigsaw-toxic-comment-classification-challenge -f '/content/drive/My Drive/ml/project/toxic/s2_2_sub.csv' -m 'sub2_2:glove840B+CNN_update'

100% 16.8M/16.8M [00:04<00:00, 4.07MB/s]
Successfully submitted to Toxic Comment Classification Challenge

In [5]:
# check competition score
!kaggle competitions submissions -c jigsaw-toxic-comment-classification-challenge

fileName           date                 description                                                                                   status    publicScore  privateScore  
-----------------  -------------------  --------------------------------------------------------------------------------------------  --------  -----------  ------------  
s2_2_sub.csv       2019-05-11 09:10:01  sub2_2:glove840B+CNN_update                                                                   complete  0.96766      0.96938       
s2_2_sub.csv       2019-05-10 23:28:39  sub2_2:glove840B+CNN_update                                                                   complete  0.96766      0.96938       
s2_2_sub.csv       2019-05-10 23:04:25  sub2_2:glove840B+CNN_update                                                                   complete  0.95929      0.96174       
s2_2_sub.csv       2019-05-10 22:47:04  sub2_2:glove840B+CNN_update                                                                   comple

#_ resouces _

> 主要参考资料：